# Module imports

In [3]:
from sympy.physics.mechanics import *
from sympy import trigsimp,symbols,cos,sin, lambdify
from math import pi,radians
import numpy as np
from scipy import linalg, integrate
init_vprinting()

# Symbolic variables

In [4]:
q0,q1,q2 = dynamicsymbols('q0 q1,q2')
#q0d,q1d,q2d = dynamicsymbols('q0 q1 q2',1)
u0,u1,u2 = dynamicsymbols('q0,q1,q2',1)

In [5]:
m0,m,I1,I2,g=symbols('m0,m,I1,I2,g')
L,R,S=symbols('L,R,S')

# Reference Frames

In [6]:
N = ReferenceFrame('N')
A = N.orientnew('A','Axis',[0,N.z])
O=Point('O')
B1=N.orientnew('B1','Axis',[-q1, N.z])
P1 = O.locatenew('P1',L*B1.y)
G1 = O.locatenew('G1',(L/2)*B1.y)
B2=B1.orientnew('B2','Axis',[-q2, B1.z])
P2 = P1.locatenew('P2',L*B2.y)
G2 = P1.locatenew('G2',(L/2)*B2.y)
P = O.locatenew('P',q0*N.x)

# Velocities

In [7]:
P.set_vel(N,u0*N.x)
A.set_ang_vel(N,0)
B1.set_ang_vel(N,-u1*N.z)
B2.set_ang_vel(N,-(u1+u2)*N.z)
O.set_vel(N,0)
P1.v2pt_theory(O,N,B1)
G1.v2pt_theory(O,N,B1)
P2.v2pt_theory(P1,N,B2)
G2.v2pt_theory(P1,N,B2)

# Kinetic and potential energies

In [8]:
Tt=(1/2)*m0*dot(P.vel(N),P.vel(N)) + (1/2)*m*dot(G1.vel(N),G1.vel(N)) +(1/2)*m*dot(G2.vel(N),G2.vel(N))
Tr=(1/2)*(m*L**2/12)*dot(B1.ang_vel_in(N),B1.ang_vel_in(N)) + (1/2)*(m*L**2/12)*dot(B2.ang_vel_in(N),B2.ang_vel_in(N))
T=Tt+Tr
#V1=m*g*dot(G1.pos_from(O),N.y)
#V2=m*g*dot(G2.pos_from(O),N.y)
#V=-R*q1+S*q2
V=0

# Constraint conditions differentiated wrt time

In [9]:
cons1=L*cos(q1)*u1+L*cos(q1+q2)*(u1+u2)-u0
cons2=-L*sin(q1)*u1-L*sin(q1+q2)*(u1+u2)

# Lagrange's equations

In [10]:
Forces=[(G1,-m*g*N.y),(G2,-m*g*N.y),(B1,-(R+S)*N.z),(B2,S*N.z)]
#Forces=[(G1,-m*g*N.y),(G2,-m*g*N.y)]
Lagrange=T-V
LM = LagrangesMethod(Lagrange, [q0,q1,q2], forcelist = Forces, nonhol_coneqs=[cons1,cons2], frame=N)
LM.form_lagranges_equations()

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                    2                                      2        L⋅g⋅m⋅(-s
⎢0.0416666666666667⋅L ⋅m⋅(2⋅q₁̈ + 2⋅q₂̈) + 0.333333333333333⋅L ⋅m⋅q₁̈ + ──────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                         2                   
⎢                                                  - 0.5⋅L ⋅m⋅(-q₁̇ - q₂̇)⋅sin
⎣                                                                             

                                                                              
                                                   

# Matrices in state space

In [11]:
mm=LM.mass_matrix_full

In [12]:
ff=LM.forcing_full

# Substitutions

In [13]:
subs_dict={L:1,m0:1,m:1,g:10,R:1,S:0}
mmq=mm.subs(subs_dict)
forcingq=ff.subs(subs_dict)

In [14]:
#mmq

In [15]:
#forcingq

# Generation functions

In [16]:
mmf=lambdify([q0,q1,q2],mmq)
forcingf=lambdify([q0,q1,q2,u0,u1,u2],forcingq)

In [17]:
#mmf(1,1,1)

In [18]:
#stv=[q0,q1,q2,u0,u1,u2]

# Function for right hand side of Runge Kutta method

In [19]:
def dydt(stv,t):
    amat=mmf(stv[0],stv[1],stv[2])
    bvec=forcingf(stv[0],stv[1],stv[2],stv[3],stv[4],stv[5])
    xvec=linalg.solve(amat,bvec)
    return xvec.flatten()

# Initial conditions and time steps

In [20]:
stv_in=np.array([1.5,radians(30),radians(60),0,0,0,0,0],dtype=float)
nval=100; ts = np.linspace(0, 1, nval)

# Integration in time

In [21]:
out=integrate.odeint(dydt,stv_in,ts)

In [22]:
out[:,7]

array([ 0.        ,  0.04786767,  0.09572134,  0.14354692,  0.19133021,
        0.23905676,  0.28671184,  0.33428036,  0.38174677,  0.42909497,
        0.47630828,  0.52336926,  0.5702597 ,  0.61696045,  0.66345134,
        0.70971106,  0.75571705,  0.80144532,  0.84687035,  0.89196493,
        0.93669998,  0.98104443,  1.02496498,  1.06842599,  1.11138928,
        1.15381395,  1.19565632,  1.23686976,  1.27740474,  1.31720892,
        1.35622744,  1.39440353,  1.43167956,  1.46799867,  1.50330757,
        1.53756075,  1.57072718,  1.60280055,  1.63381529,  1.6638714 ,
        1.69317335,  1.72209127,  1.75125764,  1.78172107,  1.81518982,
        1.85441365,  1.90376335,  1.9700484 ,  2.06348392,  2.1983176 ,
        2.39181584,  2.65961489,  3.00709407,  3.42210543,  3.87752142,
        4.34324282,  4.7970203 ,  5.22737076,  5.63098334,  6.00908543,
        6.36483131,  6.70187433,  7.02371946,  7.33348888,  7.63387877,
        7.92719285,  8.21539966,  8.50019051,  8.78302942,  9.06

# Exporting position values for animation

In [23]:
qlist=[q0,q1,q2]
plist=[O,P1,P2]
slist=out

In [24]:
xvalt=np.array([],float)
yvalt=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    xval=np.array([],float)
    yval=np.array([],float)
    for pdum in plist:
        #print(pdum)
        posv=pdum.pos_from(O).subs(subs_dict)
        #print(posv)
        for i in range(0,len(qlist)):
            posv=posv.express(N).subs({qlist[i]:slist[ii,i]})
        posvx=dot(posv,N.x).evalf()
        posvy=dot(posv,N.y).evalf()
        #print(posvx)
        xval=np.append(xval,posvx)
        yval=np.append(yval,posvy)
    xvalt=np.append(xvalt,xval)
    yvalt=np.append(yvalt,yval)
xvalt=np.reshape(xvalt,(len(slist),len(plist)))
yvalt=np.reshape(yvalt,(len(slist),len(plist)))
val=np.block([xvalt,yvalt])

In [25]:
np.savetxt('db_check_anim2.txt',val)

In [26]:
#xvalt

# Derivative of state vector with values for lambdas

In [29]:
stv_deriv=np.array([],float)
for ii in range(0,np.shape(slist)[0]):
    amat=mmf(slist[ii,0],slist[ii,1],slist[ii,2])
    bvec=forcingf(slist[ii,0],slist[ii,1],slist[ii,2],slist[ii,3],slist[ii,4],slist[ii,5])
    xvec=linalg.solve(amat,bvec)
    stv_deriv=np.append(gen_acc,xvec)
stv_deriv=np.reshape(gen_acc,(len(slist),8))

In [30]:
stv_deriv[:,6]

array([-2.71103605, -2.71148142, -2.71281493, -2.7150286 , -2.71810897,
       -2.72203675, -2.72678636, -2.73232536, -2.73861361, -2.74560232,
       -2.75323278, -2.76143492, -2.77012546, -2.77920569, -2.78855879,
       -2.79804652, -2.80750526, -2.81674109, -2.82552381, -2.83357961,
       -2.84058195, -2.8461403 , -2.84978608, -2.85095507, -2.84896522,
       -2.84298858, -2.83201542, -2.81480824, -2.78984234, -2.75522836,
       -2.70861074, -2.64703338, -2.56676036, -2.46303512, -2.32975326,
       -2.1590149 , -1.94050626, -1.66063898, -1.3013431 , -0.8383662 ,
       -0.23887327,  0.54192369,  1.56541957,  2.91579069,  4.70746437,
        7.09212949, 10.2594275 , 14.41601796, 19.7106792 , 26.05800298,
       32.85325504, 38.7714791 , 42.12270897, 41.91344466, 38.62356476,
       33.71288134, 28.5559696 , 23.92419651, 20.06839641, 16.96490858,
       14.49207814, 12.51540588, 10.91816812,  9.60771077,  8.51304901,
        7.58034611,  6.76860571,  6.04620093,  5.38826015,  4.77